In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("7480c1e7-7747-4bd2-9f27-dc599fdd1cf9")

------------

# LIGHT VALIDATION OF A MODEL PREDICTING A BINARY TARGET


## Summary

In this notebook a variety of metrics are calculated to assess the performance of a model predicting a binary outcome. <br>
We assume the model is delivered to the validator and this is taken as a starting point. <br>
As this is the most basic version of a model validation, the script does not use previously calculated object as inputs such as 'clusters', 'trained autoencoder to determine multivariate outliers', a benchmark model, nor predefined numbers on a training for example to assess the representativeness. To take these into account, (a) different specification(s) must be used. 

In the table below, we show the quantitative analyses being carried out in this script. Note that a complete Validation report also contains qualititive challenging (e.g. model assumptions, conceptual soundness, documentation etc.). These sections are also added for completeness.

| Content | Present |
| :- | :-: |
| **Data** |
| Portfolio overview | x
| Input variables accuracy | x
| Representativeness | x
| **Model performance** |
| Discriminatory power | x
| Predictive power | x
| Fairness | 
| Bias |
| Benchmark model |


## Use case

Monitoring of model performance.

When a 'date' variable will be chosen as the batchkey variable at instance creation time, this notebook shall be executed onto the different dates available in the dataset taken as input for this script. <br>
Model performance metris will be calculated for each of these dates and the result is a dataset that contains for each data all metrics of interest. This output dataset in turn can then be analyzed, i.e. the metrics can be plotted as a function of time (model monitoring). The latter will be done by means of another specification.

---

<a id="toc"></a>
## Table of Contents
1. [Setup](#set)     

    1.1 [Import Libraries](#imp_lib)  
    
    1.2 [Load input](#load_input)   
    
    1.3 [Initialize Validation Object](#init_val)    
    
2. [Documentation](#doc)

3. [Data analysis](#data_ana)       

    3.1 [Inspection of raw data](#datinsp)   
    
    3.2 [Univariate analysis of INPUT](#uni_inp)   
    
    3.3 [Univariate analysis of TARGET](#uni_tar)
    
    3.4 [(Multi-) collinearity](#corr)
    
    3.5 [Representativeness](#repr)
    
    3.5.1 [Continuous model variables](#repr_cont)
    
    3.5.2 [Categorical model variables](#repr_cat)
    
4. [Conceptual soundness](#concept) 

    4.1 [Methodology](#method) 
    
    4.2 [Modelling assumptions](#assumptions) 
    
5. [Model assessment](#model_assess)  

    5.1 [Back testing](#bactes)   
    
    5.1.1 [Binomial test - full scope](#binomial_full)   
    
    5.1.2 [Binomial test - segments](#binomial_segment) 
        
    5.2 [Discriminatory power](#disc_power)   
    
    5.2.1 [Confusion matrix](#conf_matrix)   
    
    5.2.2 [ROC and precision-recall](#roc_gini) 


<a id="set"></a>
## 1. [Setup](#toc)
---
<a id="imp_lib"></a>
### 1.1. [Import Libraries](#toc)

In [ ]:
# standard python libraries
import json, os
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.core.display import HTML
import sklearn.metrics as sk_metrics
import scipy.stats as stats
import scipy

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from yields.providers.filesystem import ProviderFactoryFilesystem #To write into filesystem
from yields.providers.hdfs import ProviderFactoryHdfs #To write into HDFS

#misc
import datetime
import statsmodels.api as sm # recommended import according to the docs

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#misc
import datetime



<a id="load_input"></a>
### 1.2. [Load Inputs](#toc)

In [ ]:
#Aliases of inputs, outputs, parameters
session.io.inputs, session.io.outputs, session.io.parameters

#### 1.2.1. [Parameters](#toc) 

In [ ]:
# user parameters to be used in this script and defined in the UI when creating the specification
GROUPBY = session.io.parameter('SEGMENT').value # bucketting attribute
THRESHOLD = session.io.parameter('THRESHOLD').value # prediction discretization threshold, ex: 0.5 ( [0, x] -> 0, (x, 1] -> 1 )

In [ ]:
print('Segmentation variable: ' + str(GROUPBY))
print('Threshold variable: ' + str(THRESHOLD))

#### 1.2.2. [Data and data model mapping](#toc)

Read dataset and model mapping

In [ ]:
input_artifact_ds = session.io.input('ds')
file_handler = input_artifact_ds.file(input_artifact_ds.files[0]).open()
raw_data = pd.read_csv(file_handler)

input_artifact_dmm_ds = session.io.input('data_model_mapping_ds')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
schema = json.load(file_handler)


#### 1.2.2. [Training dataset properties](#toc)
Read the dictionary containing the train dataset properties: this is stored inside an artifact as an output from a previous instance run. Those are numbers calculated on the training dataset and will be used here to assess the representativeness of the data between training and application dataset.

In [ ]:
input_artifact_tf = session.io.input('training_facts')
file_handler = input_artifact_tf.file(input_artifact_tf.files[0]).open()
training_props = json.load(file_handler)
    
#Show part of the file content
print('training props type:' + str(type(training_props)))

<a id="init_val"></a>
### 1.3. [Initialize Validation Object](#toc)

In [ ]:
# We are going to create a dataframe containing all our metrics of interest. Therefore, we need a structural manner to keep track of them.
# We define the fields to be saved inside the dataframe (@saving time, see end of the script, we will create the row key)
# Off course, it is up to the user to define whatever additional columns he would like to see inside this dataframe (e.g. if same metric for a benchmark model would be calculated, we could introduce another column 'model' that might take value 'under assessment' or 'benchmark')

key_name, key_value, key_meta = 'Metric_name', 'Metric_value' , 'Metric_type'
keys_ = [key_name, key_value, key_meta]
df_metrics = pd.DataFrame(columns = keys_)


<a id="doc"></a>
# 2. [Documentation](#toc)

<a id="data_ana"></a>
# 3. [Data analysis](#toc)
---

The data on which the model is applied is subject of a proper assessment. A few basic analysis (inspection of data, univariate analysis, ...) are performed to inspect the data. For a complete assessment of the data quality, we refer to the results obtained by the data quality specification. 

<a id="datinsp"></a>
## 3.1 [Inspection of raw data](#toc)



In [ ]:
#preview the data by taking a sample and check if all relevant fields (model inputs and other relevant 'context' fields) look ok. 
#In this way, for example, one can already check for some transformations that might have gone wrong.
raw_data.sample(n=10, random_state=42)

##### The dataset schema

In [ ]:
# the defined dataset schema to check for the type (finite (categorical), countable (categorical), uncountable (continuous))
#schema_view = registry.get_dataset_schema_html(config.data_set_config)
#HTML(schema_view)
target, prediction, continuous_inputs, categorical_inputs = schema['target'], schema['prediction'] ,  schema['continuous_inputs'] , schema['categorical_inputs']
print('TARGET VARIABLE: ', target)
print('MODEL PREDICTION VARIABLE: ', prediction)
print('CONTINUOUS inputs: ', continuous_inputs)
print('COUNTABLE inputs: ', categorical_inputs)

<a id="uni_inp"></a>
## 3.2 [Univariate analysis of INPUT](#toc) 

In general, we advice to keep raw_data_with_outliersa separate data quality specification in which the data quality is assessed independently from the model. <br>
Then in this notebook, we might want to answer data-related questions that might impact the overal model, e.g. (multi)collinearity and/or analyses more specifically tailored to the model at hand (e.g sensitivity analyses: what if we were to change variable x with y%, how would the model outcome change?).

### A summary of some basic statistics for the input variables - both descriptive and visual:

**Continuous input variables:**

In [ ]:
sum_stat = raw_data[continuous_inputs].describe() # calculate basic descriptive statistics 

df_outliers = pd.DataFrame(columns = ['Variable' , 'Number outliers' , '% outliers' , 'Number missings' , '% missing'])

for var in continuous_inputs:
    
    #Determine whether observations is univariate outlier (based on inter-quartile range) and add this as a flag to the dataset. afterwards, we can then use this to assess the impact of univariate outlires on model performance
    
    #Count missings
    x = raw_data[var]
    NA_count = x.isna().sum()
    NA_pct = NA_count / x.size
    x = x[~x.isna()]
    
    if len(x) == 0:
        continue
        
    #Count outliers based on IQ
    eps = 1e-100
    q1, q3 = np.quantile(x,0.25) , np.quantile(x,0.75)
    iq = q3 - q1
    lb = q1 - 3*iq - eps
    ub = q3 + 3*iq + eps
    outliers_iq = np.where((x < lb) | (x > ub) , 1, 0)
    
    number_outliers = np.sum(outliers_iq)
    N_outlier_pct = number_outliers / x.size
    
    x_no = x[(x > lb) & (x < ub)]
       
    # Fill for each variable number of outliers and percentage in a dataframe to display
    df_outliers.loc[df_outliers.shape[0]] = [var , number_outliers , N_outlier_pct , NA_count, NA_pct]
    
    fig = make_subplots(rows = 1,cols = 2, subplot_titles = ("All data" , "Outliers removed"))
    
    fig.add_trace(go.Histogram(x = raw_data[var], showlegend = False) , 1, 1)
    fig.update_xaxes(title_text = var, row = 1, col = 1)
    fig.update_yaxes(title_text = 'Counts' , row = 1, col = 1)
    
    fig.add_trace(go.Histogram(x = x_no, showlegend = False) , 1, 2)
    fig.update_xaxes(title_text = var, row = 1, col = 2)
    fig.update_yaxes(title_text = 'Counts' , row = 1, col = 2)
    fig.show()

#Append the number of outliers for each variable to the general descriptive statistics.    
df_outliers = df_outliers.set_index('Variable').transpose()
sum_stat = pd.concat([sum_stat, df_outliers])

# We want to keep track of the descriptive statistics of every raw variable to plot the time evolution in a second step. We therefore need to add these to the metrics dataframe;
#In this way, we can see how e.g. the mean/median of these variables are changing as time passes. This in turn might indicate shifts in the data that might impact the model performance.

for var in continuous_inputs:
    for stat in sum_stat.index:
        #Add each metric to the bottom of the df_metrics dataframe
        df_metrics.loc[df_metrics.shape[0]] = [var + '_' + stat , sum_stat[var][stat] , 'DataQuality_Continuous'] #Should follow schema as defined by df_metrics, i.e. 'Metric_name', 'Metric_value' , 'Other_Metadata'
        
#some styling to finally display the descriptive statistics.
sum_stat_st = sum_stat.transpose().style.set_caption('Descriptive statistics of continuous model variables').format({"% outliers" : "{:.2%}" , "% missing" : "{:.2%}"})
display(sum_stat_st)        

        

**Finite input variables:**

In [ ]:
# Here we give an overview of some basic statistics for each finite input variable in the data set
sum_stat = raw_data[categorical_inputs].dropna(axis=0).describe() # calculate basic statistics 
display(sum_stat) #show the descriptive statistics dataframe

for var in categorical_inputs:
    for stat in sum_stat.index:
        #Add each matric to the bottom of the df_metrics dataframe
        df_metrics.loc[df_metrics.shape[0]] = [var + '_' + stat , sum_stat[var][stat] , 'DataQuality_Categorical']
        
    #Also, display the proportion of observations per category for each variable
    x = raw_data[var]
    NA_count = x.isna().sum()
    NA_pct = NA_count / x.size
    x = x[~x.isna()]
    
    var_df = pd.DataFrame(columns = ['Category' , 'Number observations' , 'Proportion (%)'])
    [uni, counts] = np.unique(x,return_counts = True)
    var_df['Category'] = uni
    var_df['Number observations'] = counts
    var_df['Proportion (%)'] = counts / len(x)
    
    display(var_df.set_index('Category').style.set_caption(f'Overview for <b>{var}</b>').format({'Proportion (%)' : "{:.2%}"}))
        


<a id="uni_tar"></a>
## 3.3 [Univariate analysis of TARGET](#toc) 

Here we inspect the main indicator we are interested at in this data set: the number of events whose occurrence is supposed to be modelled. We look first at the overall number of events and then at its decomposition along a bucketting attribute.

In [ ]:
# Get the overall positive target rate. 
# Target obtained through schema by calling the 'target' attribute, i.e. schema.target
# define some auxiliary variables
nbr_targets , nbr_targets_val = "Number_positive_targets" , raw_data[target].sum() #Number of observations with 'target = 1'
nbr_obs , nbr_obs_val = "Number_of_Observations" , len(raw_data)
target_rate , target_rate_val = "Observed_Target_Rate" , raw_data[target].mean()

df_dict = {nbr_targets: nbr_targets_val , nbr_obs : nbr_obs_val , target_rate : target_rate_val}

#Make dataframe for visualization
display(pd.DataFrame(df_dict , index = [0]))

#Store numbers in metrics dataframe
for key in df_dict.keys():
    df_metrics.loc[df_metrics.shape[0]] = [key , df_dict[key]  , 'Portfolio description']
    

In [ ]:
# if groupby variable is defined, we are also interested in the target rate per bucket. 
# Only makes sense if groupby is considered categorical (for continuous variables, one might create its own categorization to get proper insights) 
if GROUPBY != None and GROUPBY in categorical_inputs: #one can do this for all categorical inputs - see also next cell.
    
    tmp = raw_data[[GROUPBY,target]]
    
    tmp_grp = tmp.groupby(GROUPBY)[target].agg(['sum', 'count', 'mean'])
        
    print('summary statistics on the observed target rate for variable ' + GROUPBY)
    display(tmp_grp)

    #Store numbers in metrics dataframe
    for index in tmp_grp.index:
        df_metrics.loc[df_metrics.shape[0]] = ['Observed_Target_Rate_' + GROUPBY + '_' + str(index) , tmp_grp.loc[index]['mean']  , 'Portfolio description']


In [ ]:
# One can visualize the observed target rate for every categorical input variable: this gives insights already insights on how well these variable might be able to predict

for cat in categorical_inputs:
    
    tmp = raw_data[[cat,target]].astype('double') #to double to drop missing values
    tmp[target] = tmp[target]
    tmp = tmp.dropna()
    
    tmp_grp = tmp.groupby(cat)[target].agg('mean')
    
    #For each categorical input variable: create histogram that shows observed target rate for each category
    fig = make_subplots()
    fig.update_layout(title_text = 'Observed target rate by ' + cat)
    fig.update_xaxes(title_text = cat)
    fig.update_yaxes(tickformat = ',.1%') #observed rate is a percentage, so show it like that (up to 1 decimal number after comma).
    fig.add_trace(go.Bar(x=tmp_grp.index , y = tmp_grp.values , name = 'Group observed target rate') )
    fig.add_trace(go.Scatter(x = tmp_grp.index , y = target_rate_val*np.ones(len(tmp_grp.index)) , name = 'Total observed target rate'))
    fig.show()

<a id="corr"></a>
## 3.4 [(Multi-) collinearity](#toc)

We will assess the following correlations to analyse potential correlation, that might impact the overal model's stability and/or violates potential assumptions.

1. for continuous variables: Spearman correlation
2. for categorical variables: Cramer's V
3. We also analyse the amount of multicollinearity, meaning we look for 'one independent variable being potentially explained by another independent variable'. This implies the independent variables are not truly independent and this can cause overal model instability and causes potential wrong statistical inference when e;g building the model (having multicollinearity has as a consequence that the variance of the estimates coefficients is underestimated. This in turn impacts it's p-value and hence whether it should be retained (or not))

In [ ]:
"""CONTINUOUS MODEL INPUTS"""

thresh = 0.5 # threshold to select 'correlated' variables
import plotly.figure_factory as ff

def cramers_v(x, y):
    
    confusion_matrix = pd.crosstab(x,y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    cramersV = np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))
    
    return cramersV

df_correlated = pd.DataFrame(columns = ["Test", "var 1", "var 2", "r"])
                   
#For continuous variables, let's use spearman correlation (distribution-free meausure)
df_ = raw_data[continuous_inputs]
                                        
corr_mat = df_.corr(method='spearman')
        
z_ = corr_mat.values
z_text = np.round(z_,2)
x_ = list(df_.columns)
y_ = list(df_.columns)

# make selection of moderately and highly correlated variables\n",
for j in np.arange(len(x_)):
    for k in np.arange(j+1,len(y_)):
       if np.abs(z_[j,k]) > thresh:
            df_correlated.loc[len(df_correlated)] = ['Pearson', x_[j], y_[k], np.round(z_[j,k],2)]

df_correlated = df_correlated.sort_values(by=['r'], ascending=False)

fig = ff.create_annotated_heatmap(z = z_, annotation_text = z_text, x = x_, y = y_, colorscale='Viridis')
fig['layout']['yaxis']['autorange'] = "reversed"
fig['layout']['margin'] = {'t' : 300}
fig['layout']['title'] = {'text' : 'Feature Correlation of continuous risk drivers by Pearson correlation'}
fig['layout']['height'] = 900
fig['layout']['width'] = 900
fig.show()

In [ ]:
"""CATEGORICAL MODEL INPUTS"""

thresh = 0.3 # threshold to select 'correlated' variables
import plotly.figure_factory as ff

def cramers_v(x, y):
    
    confusion_matrix = pd.crosstab(x,y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    cramersV = np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))
    
    return cramersV

df_correlated = pd.DataFrame(columns = ["Test", "var 1", "var 2", "r"])
                   
#For continuous variables, let's use spearman correlation (distribution-free meausure)
df_ = raw_data[categorical_inputs].astype('double')
                                       
corr_mat = df_.corr(method= cramers_v)

display(corr_mat.style.set_caption('Feature Correlation of categorical risk drivers by Cramers V correlation'))
        

<a id="repr"></a>
# 3.5 [Representativeness](#toc) 

<a id="repr_cont"></a>
## 3.5.1 [Continuous model variables](#toc) 

We previously saved the continuous model variables inside a dictionary. The distribution of each variable can be seen as a sample from some unknown distribution. This sample serves as one ingredient to perform a two-sample Kolmogorov-Smirnov test. The second sample is the observed distribution of the same variable but on the application dataset. <br>
This KS-test performs a statistical test in which the null-hypothesis is that 'the two samples are generated from the same underlying distribution'. In other words, if we find a significant p-value, 
we have some evidence that the data on which we assess the model in this notebook (which is the application dataset (could be out-of-time, out-of-sample or any other) is no longer similar to the training dataset.<br>
This decreased representativeness in turn might lead to an increase risk of bias of the model outcome. How this potential bias harms the model and to what extent needs further (use-case specific) investigation, as the followings tests only serve as a smoking gun for potential representativeness issues.

https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test

In [ ]:
#let's do a KS-test for every continuous model input variable

for var in continuous_inputs:
    
    #Retrieve training sample from the dictionary that we loaded in the beginning and let's make an numpy array from it (as this is what the KS test needs in python). We saved it however as a dictionary.
    sample_training = np.fromiter(training_props['Continuous_training'][var].values() , dtype = 'float')
    sample_test = raw_data[var].values #Raw data is the dataset loaded in this notebook.
    KS_stat, KS_pval = scipy.stats.ks_2samp(sample_training, sample_test, alternative='two-sided', mode='auto')

    #Store both the KS_statistic and the p-value inside the metrics dataframe for monitoring purposes.
    df_metrics.loc[df_metrics.shape[0]] = [var + '_KS_statistic' , KS_stat , 'Representativenesss_Continuous']
    df_metrics.loc[df_metrics.shape[0]] = [var + '_KS_pvalue' , KS_pval , 'Representativenesss_Continuous']
    
    #Let's plot the training data and application data on the same figure, both the probability distribution and the cumulative distributions (which is the basis for the KS-test).
    fig = make_subplots(rows = 1,cols = 2, subplot_titles = ("Probability distribution" , "Cumulative prob. distr."))
    fig.add_trace(go.Histogram(x = sample_training , name = 'training', histnorm = 'probability density', showlegend = False), 1, 1)
    fig.add_trace(go.Histogram(x = sample_test , name = 'test' , histnorm = 'probability density' , showlegend = False) , 1, 1)
    fig.update_traces(opacity=0.6)
    fig.update_xaxes(title_text = 'Domain ' + var , row = 1, col = 1)
    fig.update_yaxes(title_text = 'Frequency' , row = 1, col = 1)
    fig.update_layout(barmode='overlay')
    
    ecdf_training = sm.distributions.ECDF(sample_training)
    x_training = np.linspace(min(sample_training), max(sample_training))
    fig.add_trace(go.Scatter(x = x_training, y = ecdf_training(x_training) , name = 'training', line=dict(color='blue')), 1, 2)

    ecdf_test = sm.distributions.ECDF(sample_test)
    x_test = np.linspace(min(sample_test), max(sample_test))
    fig.add_trace(go.Scatter(x = x_test, y = ecdf_test(x_test) , name = 'test' , line=dict(color='red')) , 1, 2)
    fig.update_xaxes(title_text = 'Domain ' + var , row = 1, col = 2)
    fig.update_yaxes(title_text = 'CDF' , row = 1, col = 2)
    fig.show()

<a id="repr_cat"></a>
## 3.5.2. [Categorical model variables](#toc) 

We previously saved for each categorical model variables it's distribution over it's categories on the training dataset inside a dictionary. 
We will now calculated the same proportion per category for each model variabel and compare it with the one from the training dataset. <br>
The measure used to quantify a possible shift is the so-called Population Stability Index. For a single variable, it is calculated as follows:

$PSI(train, test) = \sum \limits _{k=1} ^{N} [Proportion_{test}(k) - Proportion_{train}(k)] \times  ln \frac{Proportion_{test}(k)}{Proportion_{train}(k)}$

where the sum over k runs over all categories present in the given variable. All we need is the proportion of observations within each category. <br>

Note that the $PSI \geq 0$ where it is only exactly 0 when the two discrete distributions are exactly the same.


https://scholarworks.wmich.edu/cgi/viewcontent.cgi?article=4249&context=dissertations

In [ ]:
for var in categorical_inputs:

    #Retrieve proportions of the training dataset (these have been calculated and stored previously as to avoid data leakage between train and test set). It also allows a single ground truth to share some training set metrics.
    prop_training = training_props['Categorical_training'][var]
    keys_values = prop_training.items()
    prop_training = {int(key): value for key, value in keys_values}
    cats_training = list(prop_training.keys())

    #Calculate proportions pre category on application dataset ('raw data')
    x = raw_data[var]
    x = x[~x.isna()]
    [uni, counts] = np.unique(x,return_counts = True) #this always gives a SORTED (sorted by category number). The ordering should be checked as we should definitely compare the proportions within the same category of the given variable.
    prop = counts / len(x)
    uni = uni.tolist()

    df = pd.DataFrame(data = prop_training ,index=['Train']).T


    if cats_training == uni: #PSI can only be calculated if exactly the same categories are present in both datasetse (we compare to ordered lists). Extra categories appearing might indicate either 1) categories unoccupied during training 2) new categories have been added (e.g. process related). Both will throw appropriate warnings are give a high PSI meaning large shift

    #Again, it's important that we do not do operations that might change the order of the categories. Most likely, if we made some mistake it can be easily spotted as we observe unexpected changes.
        
        df = pd.DataFrame(data = prop_training ,index=['Train']).T
        df['Test'] = prop    
        df['PSI'] = (df['Test'] - df['Train']) * np.log(df['Test'] / df['Train'])
        
        
        #Visualization: plot discrete distributions of train and application set.
        fig = make_subplots()
        fig.update_layout(title_text = ' Discrete prob. distribution of variable  ' + cat)
        fig.add_trace(go.Bar(x=df.index , y = df['Train'].values , name = 'Train') )
        fig.add_trace(go.Bar(x=df.index , y = df['Test'].values , name = 'Test') )
        fig.show()
        
        df.loc['Total'] = np.sum(df,axis=0)
        print(var)
        display(df)
        
        #Add total PSI to metrics dataframe.
        df_metrics.loc[df_metrics.shape[0]] = [var + '_PSI' , df.loc['Total','PSI'] , 'Representativenesss_Categorical']
        
     
    else:
        
        #Add total PSI (Not-a-number) to metrics dataframe.
        df_metrics.loc[df_metrics.shape[0]] = [var + '_PSI' , np.nan , 'Representativenesss_Categorical']
        
        print("For variable " + var + " the number of categories is different as compared to the trainset. Impossible to calculate PSI")
        


<a id="concept"></a>
# 4. [Conceptual soundness](#toc) 

<a id="method"></a>
# 4.1 [Methodology](#toc) 

Here the model assessor should give an descriptive overview on the model methodology, e.g.

* binary target
* logistic regression
* 10 variables used:
    * variable 1: meaning + does the model behave intuitive given this variable. For example, increasing/decreasing its value should increase/decrease the probability
    * variable 2: meaning
    * ...

<a id="assumptions"></a>
# 4.2 [Modelling assumptions](#toc) 

This section may contain another set of model expectations that we shall validate and/or monitor. These expectations should here be less 'business driven' and should instead focus on the 'technical' or data driven expectations we have about our model. They may apply on the model input (stationarity, gaussianity, etc.) or output (error distribution, etc.).

<a id="model_assess"></a>
# 5. [Model assessment](#toc) 

<a id="bactes"></a>
## 5.1 [Back testing](#toc) 

<a id="binomial_full"></a>
### 5.1.1 [Binomial test - full scope](#toc) 

In a binomial test, we compare the observed number of targets == 1 with the predicted target rate. The tests looks for statistical significant differences, i.e. "is there enough evidence that should make us believe the observed deviation between prediction and observation is not a coincidence?". Stated otherwise: if we find a significant p-value, we might believe the observed devation is not because of random effects, but it is systematically.

Note, however, that if the result is statistically significant ("we assume there is systematic deviation between prediction and observation"), the size of this different (so-called effect size) might not be practically significant. So we need both.

In order to perform a binomial test we need some predicted rate. Because the current dataset (ppnr) assigns an indivudual estimate for each observation, we define the overall estimate to the the average of all these estimates.

In [ ]:
# Let's create a dataframe to nicely display the results.
df_binom = pd.DataFrame(columns = ['Ptf' , 'Number of observations' , 'Number of events' , 'Observed event rate' , 'Estimated event rate' ,'p-value', 'Effect size (Cohen h)'])

# Define the overall prediction rate as the mean of the indvidual estimates. It is against this value we shall assess the realised numbers of targets = 1.
prob_overall = np.mean(raw_data[prediction])

# Statistical test (binomial)
NBR_TARGET = len(raw_data.loc[raw_data[target] == 1])
N_OBS = len(raw_data)
p_val_binom = stats.binom_test(NBR_TARGET, N_OBS, prob_overall, alternative='two-sided')

#Effect size - https://en.wikipedia.org/wiki/Cohen%27s_h
PD_obs = NBR_TARGET / N_OBS
cohen_h = 2*np.abs(np.arcsin(np.sqrt(prob_overall)) - np.arcsin(np.sqrt(PD_obs)))

# Fill in the dataframe
df_binom.loc[len(df_binom)] = ['Full dataset' , N_OBS , NBR_TARGET , NBR_TARGET / N_OBS , prob_overall , p_val_binom, cohen_h]
display(df_binom)

# Print the conclusion
print('p-value binomial test: ' + str(p_val_binom))

alfa = 0.05 #this parameter can be defined as such in the UI and be retrieved here.
if p_val_binom < alfa:
    print('The difference between observed target rate and predicted target rate is statistically significant at the ' + str(alfa) + ' level.')
elif p_val_binom >= alfa:
    print('The difference between observed target rate and predicted target rate is NOT statistically significant at the ' + str(alfa) + ' level.')
    
#Finally, let's add this p-value to the metrics, so that we can track through time what will bese p-values
df_metrics.loc[df_metrics.shape[0]] = ['p_value_binomial_full_dataset' , p_val_binom , 'Backtest' ]
df_metrics.loc[df_metrics.shape[0]] = ['effect_size_binomial_full_dataset' , cohen_h , 'Backtest' ]

<a id="binomial_segment"></a>
### 5.1.2 [Binomial test - segments](#toc) 

Same as previous but now run over all categories in the variable of interest (GROUPBY parameter).

In [ ]:
# Let's create a dataframe to nicely display the results.
df_binom_seg = pd.DataFrame(columns = ['Ptf' , 'Number of observations' , 'Number of events' , 'Observed event rate' , 'Estimated event rate' ,'p-value', 'Effect size (Cohen h)'])

# Define the overall prediction rate as the mean of the indvidual estimates. It is against these values we shall assess the realised numbers of targets = 1.

cats = raw_data[GROUPBY].unique()

for var in cats:

    data_tmp = raw_data[raw_data[GROUPBY] == var]
    
    prob_overall = np.mean(data_tmp[prediction])

    # Statistical test (binomial)
    name = GROUPBY + '_' + str(var)
    NBR_TARGET = len(data_tmp.loc[data_tmp[target] == 1])
    N_OBS = len(data_tmp)
    p_val_binom = stats.binom_test(NBR_TARGET, N_OBS, prob_overall, alternative='two-sided')
                                       
     #Effect size - https://en.wikipedia.org/wiki/Cohen%27s_h
    PD_obs = NBR_TARGET / N_OBS
    cohen_h = 2*np.abs(np.arcsin(np.sqrt(prob_overall)) - np.arcsin(np.sqrt(PD_obs)))                                  

    # Fill in the dataframe
    df_binom_seg.loc[len(df_binom_seg)] = [name , N_OBS , NBR_TARGET , NBR_TARGET / N_OBS , prob_overall , p_val_binom , cohen_h]
    
    #Finally, let's add this p-value to the metrics, so that we can track through time what will bese p-values
    df_metrics.loc[df_metrics.shape[0]] = ['p_value_binomial_' + name , p_val_binom , 'Backtest' ]
    df_metrics.loc[df_metrics.shape[0]] = ['effect_size_binomial_' + name , cohen_h , 'Backtest' ]
    
    
display(df_binom_seg.style.set_caption('Two-sided binomial test for statistical significance, including effectd size'))

display(df_binom_seg.sort_values(['Effect size (Cohen h)' , 'p-value']).style.set_caption('Ordered by Effect size and statistical significance'))


<a id="disc_power"></a>
## 5.2 [Discriminatory power](#toc) 

<a id="conf_matrix"></a>
### 5.2.1 [Confusion matrix](#toc) 


In [ ]:
# In a confusion matrix we analyze the number of true/false positive/negatives. In order to do so, we need to define at cut-off above which our estimated outcome at individual level is believe to be a 'positive' outcome.
# This means all probabilities > THRESHOLD get the value 1. 

#Transform continuous estimate into binary by using the THRESHOLD
y_true = raw_data[target]
y_pred = [int(i > THRESHOLD) for i in raw_data[prediction]]
cm = sk_metrics.confusion_matrix(y_true, y_pred)
sk_metrics.ConfusionMatrixDisplay(confusion_matrix=cm).plot()

<a id="roc_gini"></a>
### 5.2.2 [ROC and precision-recall](#toc) 

To get an idea on how well our model predicting the target is able to discriminate between target = 0 and 1 we will show for example the ROC-curve.
This is a parametrized curve - with the THRESHOLD as parameter and plots 1-specificty vs the sensitivity.
The figure can also be aggregated into a single metric called the area-under-the-curve, with the following limiting behaviour:
AUC = 100%: perfect discrimination. The decision boundary by the model does not misclassify any observation.
AUC = 50%: our model behaves as a random classifier (i.e. label the prediction as '1' with 50% or '0' with 50% at a random. Useless model.

In [ ]:
""" ROC """

fpr, tpr, _ = sk_metrics.roc_curve(raw_data[target],  raw_data[prediction])

fig = make_subplots()
fig.update_layout(title_text = 'ROC-curve')
fig.update_xaxes(title_text = 'TPR')
fig.update_yaxes(title_text = 'FPR')
fig.add_trace(go.Scatter(x = fpr , y = tpr  , name = 'current model'))
fig.add_trace(go.Scatter(x = [0,1] , y = [0,1] , name = 'random' ))
fig.show()

""" PRECISION - RECALL """

precision , recall , _ = sk_metrics.precision_recall_curve(raw_data[target],  raw_data[prediction])

fig = make_subplots()
fig.update_layout(title_text = 'PR-curve')
fig.update_xaxes(title_text = 'Precision')
fig.update_yaxes(title_text = 'Recall')
fig.add_trace(go.Scatter(x = recall , y = precision  , name = 'current model'))
fig.show()


In [ ]:
auc_score = sk_metrics.roc_auc_score(raw_data[target],  raw_data[prediction])
print('Area-under-the-ROC-curve: ' + str(auc_score))

avg_p = sk_metrics.average_precision_score(raw_data[target],  raw_data[prediction])
print('Average precision: ' + str(avg_p))

#Finally, let's add this p-value to the metrics, so that we can track through time what will bese p-values
df_metrics.loc[df_metrics.shape[0]] = ['AUC_full_dataset' , auc_score , 'Discriminatory Power' ]
df_metrics.loc[df_metrics.shape[0]] = ['PR_full_dataset' , avg_p , 'Discriminatory Power' ]

In [ ]:
""" ROC for different segments """

cats = raw_data[GROUPBY].unique()

fig = make_subplots()
fig.update_layout(title_text = 'ROC-curve')
fig.update_xaxes(title_text = 'TPR')
fig.update_yaxes(title_text = 'FPR')
fig.add_trace(go.Scatter(x = [0,1] , y = [0,1] , name = 'random' ))
    
for var in cats:

    data_tmp = raw_data[raw_data[GROUPBY] == var]
    
    n_pos = len(data_tmp[data_tmp[target]==1])
    
    if (n_pos > 0) & (len(data_tmp[prediction]) > 0): #both prediction and target should contain values, otherwise not possible to plot roc-curve. Particular useful check when few to zer obsevations are expected in a singel category of groupby variable
    
        fpr, tpr, _ = sk_metrics.roc_curve(data_tmp[target],  data_tmp[prediction])

        name = GROUPBY + '_' + str(var)
        fig.add_trace(go.Scatter(x = fpr , y = tpr  ,  name = name ))

        auc_score = sk_metrics.roc_auc_score(data_tmp[target], data_tmp[prediction])
        df_metrics.loc[df_metrics.shape[0]] = ['AUC_' + name, auc_score , 'Discriminatory Power' ]  
        
    
fig.show()

## Save output

Now that we have taken a template (analysis specification) and filled in the code fully defining our validation document (analysis instance), we are ready to save the whole thing for future use, when new data becomes available.

In [ ]:
#For the sake of demonstration, we also show how to save the resulting csv onto both the local filesystem as well as HDFS.

output_artifact = session.io.output("vanilla_binary_medium_metrics")

#Provide some filename
import datetime
current_datetime = datetime.datetime.now()
str_cdt = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
filename = str_cdt + "model_metrics.csv"

# #Declare an output artifact version that the user can write to
output_artifact_version = output_artifact.file(filename,ProviderFactoryFilesystem())

# # Save the DataFrame to a CSV file into the above file
df_metrics.to_csv(output_artifact_version, index=False) 

output_artifact.save()